## 207 -Applied Machine Learning Project: Predicting Attrition of an Online Store Site

#### Authors:

Diego Moss
Sammy Cayo
Conor Huh
Roz Huang
Jasmine Lau



## Loading Data from BigQuery and Binding Rows

In [ ]:
## code block for initial data loading

## Initial Data Pre-Processing and Feature Extraction

In [ ]:
## code block for preprocessing

## after binding rows we will need to extract the data from the columns with multiple data and put into their own columns

## we need to manipulate and aggregate each user's data for each month for all the manipulated features in our list

## we should take a look at how bad we're missing data and decide for imputation or removal




## shaping the data in the right way for the LSTM model (3 dimensions)

def create_sequences(df, sequence_length):
    sequences = []
    targets = []
    user_ids = df['user_id'].unique()
    
    for user_id in user_ids:
        user_data = df[df['user_id'] == user_id]
        user_data = user_data.sort_values(by='month')  # Ensure data is sorted by time
        for i in range(len(user_data) - sequence_length):
            seq = user_data.iloc[i:i+sequence_length].drop(columns=['user_id', 'days_until_next_visit']).values # target can change depending on our attrition statistics
            target = user_data.iloc[i+sequence_length]['days_until_next_visit'] # target can change depending on our attrition statistics
            sequences.append(seq)
            targets.append(target)
    
    return sequences, np.array(targets)

# 12 months
sequence_length = 12

# creating our sequences and targets for putting into X and y objects
data_sequences, targets = create_sequences(df, sequence_length)

# Pad sequences to ensure all have the same length
data_sequences_padded = pad_sequences(data_sequences, maxlen=sequence_length, padding='post', dtype='float32')

# Convert to arrays and saving as X and y
X = np.array(data_sequences_padded)
y = np.array(targets)



## Splitting Data

In [ ]:
## splitting data code block

## need to see distribution of target variable and distribution of months in the data. If unequal need to sample so that the training set has roughly equal targets and months

## may need to manually split the data, make sure we are splitting by user and not by rows


## More Data Processing and EDA

In [ ]:
## EDA - visualizing distributions and correlations



## standardizing features based on the training distributions

# Standardize the features (optional)
scaler = StandardScaler()
# fitting on 2d reshaped X_train because the scaler won't work on 3 dimensions, then returning back into 3 dimensions
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[-1])).reshape(X_train.shape) 
# doing same for test set
X_test = scaler.transform(X_test.reshape(X_train.shape[0]*X_train.shape[1], X_test.shape[-1])).reshape(X_test.shape)



## Model Fitting and Architecture

In [ ]:
## Placeholder

## Hyperparameter Tuning

In [ ]:
## placeholder

## we need to see how long it takes to fit one epoch, then do some math to see how many sets of hyperparameters we can test

## Evaluation Metrics

In [ ]:
## placeholder